In [1]:
import openmc

In [2]:
# to make the geometry, convert the serpent geometry then do the following:
# 1. delete cell 334 and 335
# 2. chainge remove material from cell 333 and make the fill universe 24
# 3. make the region of surface 948 the negative halfspace

# broken geometry universes: 7,13 2, 

In [3]:
mat = openmc.Materials.from_xml()
#geo = openmc.Geometry.from_xml(path='geometry/msbr_full.xml', materials=mat)

In [4]:
colormap = {mat[0]: 'purple',
           mat[1]: 'blue',
           mat[2]: 'yellow'}

In [5]:
from copy import deepcopy
import numpy as np
moder, hast, fuel = mat

pin_bound = openmc.rectangular_prism(5.08*2, 5.08*2) # Pin outer boundary

pin_gr_sq = openmc.rectangular_prism(4.953*2, 4.953*2) # Graphite square

# params for main pin section for both I-A and II-A
r_d = 0.66802
l1 = 4.28498
l2 = 4.53898
l3 = 5.62102
lu = openmc.ZCylinder(-l1, l2, r_d)
rb = openmc.ZCylinder(l1, -l2, r_d)
lb = openmc.ZCylinder(-l2, -l1, r_d)
ru = openmc.ZCylinder(l2, l1, r_d)
lu_adj = openmc.ZCylinder(-l1, -l3, r_d)
rb_adj = openmc.ZCylinder(l1, l3, r_d)
ru_adj = openmc.ZCylinder(-l3, l1, r_d)
lb_adj = openmc.ZCylinder(l3, -l1, r_d)

pin_gr_sq_surfs = pin_gr_sq.get_surfaces()
pin_gr_sq_keys = list(pin_gr_sq_surfs.keys())
pin_gr_sq_minx = pin_gr_sq_surfs[pin_gr_sq_keys[0]]
pin_gr_sq_maxx = pin_gr_sq_surfs[pin_gr_sq_keys[1]]
pin_gr_sq_miny = pin_gr_sq_surfs[pin_gr_sq_keys[2]]
pin_gr_sq_maxy = pin_gr_sq_surfs[pin_gr_sq_keys[3]]

pin_gr_sq_pos = -pin_gr_sq_minx | +pin_gr_sq_maxx | -pin_gr_sq_miny | +pin_gr_sq_maxy
pin_gr_sq_neg = pin_gr_sq

pin_corners = pin_bound & (-lu |
                           -rb |
                           -lb |
                           -ru |
                           -lu_adj |
                           -rb_adj |
                           -ru_adj |
                           -lb_adj)

In [6]:
# Zone I-A

### REWRITE STRATEGEY
# 1. make cells
# 2. translate their regions
# 3. bound the regions with the appropriate bounding planes

# I-A 1 (lower 1)
ia_gr_round_1 = openmc.ZCylinder(r=4.953)
ia_fuel_hole = openmc.ZCylinder(r=1.71069)
ia_fuel_inner_1 = openmc.Cell(fill=fuel, region=(-ia_fuel_hole))
ia_moderator_1 = openmc.Cell(fill=moder, region=(+ia_fuel_hole &
                                                 -ia_gr_round_1))
ia_fuel_outer_1 = openmc.Cell(fill=fuel, region=(+ia_gr_round_1 &
                                                 pin_bound))
ia1 = (ia_fuel_inner_1, ia_moderator_1, ia_fuel_outer_1)

# I-A  main (lower 2)
ia_fuel_inner_main = ia_fuel_inner_1.clone(clone_materials=False)
ia_moderator_main = openmc.Cell(fill=moder, region=(+ia_fuel_hole &
                                      pin_gr_sq_neg |
                                      pin_corners))
ia_fuel_outer_main = openmc.Cell(fill=fuel, region=(pin_gr_sq_pos &
                                     pin_bound &
                                     +lu &
                                     +rb &
                                     +lb &
                                     +ru &
                                     +lu_adj &
                                     +rb_adj &
                                     +ru_adj &
                                     +lb_adj))
iam = (ia_fuel_inner_main, ia_moderator_main, ia_fuel_outer_main)

# I-A 2 (upper 1)
ia_fuel_inner_2 = ia_fuel_inner_1.clone(clone_materials=False)
ia_moderator_2 = ia_moderator_1.clone(clone_materials=False)
ia_fuel_outer_2 = ia_fuel_outer_1.clone(clone_materials=False)
ia2 = (ia_fuel_inner_2, ia_moderator_2, ia_fuel_outer_2)
                              
# I-A 3 (upper 2)
h = 12.66
theta = np.arctan(4.953 / h)
r2 = (1 / np.cos(theta))**2 - 1
cone_i = openmc.ZCone(z0=h, r2=r2)
ia_fuel_inner_3 = ia_fuel_inner_1.clone(clone_materials=False)
ia_moderator_3 = openmc.Cell(fill=moder, region=(+ia_fuel_hole &
                                                 -cone_i))
ia_fuel_outer_3 = openmc.Cell(fill=fuel, region=(+cone_i & 
                                                 pin_bound))
ia3 = (ia_fuel_inner_3, ia_moderator_3, ia_fuel_outer_3)                             

# I-A and II-A 4 (upper 3)
ia_gr_round_4 = openmc.ZCylinder(r=2.2225)
ia_hast = openmc.Cell(fill=hast, region=(-ia_fuel_hole))
ia_moderator_4 = openmc.Cell(fill=moder, region=(+ia_fuel_hole &
                                                 -ia_gr_round_4))
ia_fuel_outer_4 = openmc.Cell(fill=fuel, region=(+ia_gr_round_4 &
                                                 pin_bound))
ia4 = (ia_hast, ia_moderator_4, ia_fuel_outer_4)


# II-A main (lower 1)
iia_fuel_hole_main = openmc.ZCylinder(r=3.302) # Hole with fuel salt - Fig 3.5, Roberton 1971 (3.27787 - p.47)
iia_fuel_inner_main = openmc.Cell(fill=fuel, region=(-iia_fuel_hole_main))
iia_moderator_main = openmc.Cell(fill=moder, region=(+iia_fuel_hole_main &
                                      pin_gr_sq_neg |
                                      pin_corners))
iia_fuel_outer_main = ia_fuel_outer_main.clone(clone_materials=False)
iiam = (iia_fuel_inner_main, iia_moderator_main, iia_fuel_outer_main)

# II-A 2 (upper 1)
iia_fuel_hole_2 = openmc.ZCylinder(r=0.635)
iia_fuel_inner_2 = openmc.Cell(fill=fuel, region=(-iia_fuel_hole_2))
iia_moderator_2 = openmc.Cell(fill=moder, region=(+iia_fuel_hole_2 &
                                      pin_gr_sq_neg |
                                      pin_corners))
iia_fuel_outer_2 = ia_fuel_outer_main.clone(clone_materials=False)
iia2 = (iia_fuel_inner_2, iia_moderator_2, iia_fuel_outer_2)

# II-A 3 (upper 2)
iia_gr_round_3 = openmc.ZCylinder(r=3.65125)
iia_fuel_inner_3 = iia_fuel_inner_2.clone(clone_materials=False)
iia_moderator_3 = openmc.Cell(fill=moder, region=(+iia_fuel_hole_2 &
                                                  -iia_gr_round_3))
iia_fuel_outer_3 = openmc.Cell(fill=fuel, region=(+iia_gr_round_3 &
                                                  pin_bound))
iia3 = (iia_fuel_inner_3, iia_moderator_3, iia_fuel_outer_3)

# II-A 4 (upper 3)
h = 6.5
theta = np.arctan(3.65125 / h)
r2 = (1 / np.cos(theta))**2 - 1
cone_ii = openmc.ZCone(z0=h, r2=r2)
iia_moderator_4 = openmc.Cell(fill=moder, region=(-cone_ii))
iia_fuel_outer_4 = openmc.Cell(fill=fuel, region=(+cone_ii &
                                                  pin_bound))
iia4 = (iia_moderator_4, iia_fuel_outer_4)

# II-A 5 (upper 4)
iia_moderator_5 = openmc.Cell(fill=moder, region=(-ia_gr_round_4))
iia_fuel_outer_5 = openmc.Cell(fill=fuel, region=(+ia_gr_round_4 &
                                                 pin_bound))
iia5 = (iia_moderator_5, iia_fuel_outer_5)


            # ia   iia
pin_cells = [(ia1, iiam),
             (iam, iia2),
             (ia2, iia3),
             (ia3, iia4),
             (ia4, iia5)]
pin_levels = [(0.0, 0.0),
              (22.86, 434.34),
              (419.10, 436.88),
              (438.15, 439.42),
              (445.135, 441.96),
              (449.58, 449.58)]
#bounding surfaces
level_bounds = []
for levels in pin_levels:
    ia_bound = openmc.ZPlane(z0=levels[0])
    iia_bound = openmc.ZPlane(z0=levels[1])
    level_bounds.append((ia_bound, iia_bound))
ia_cell_list = []
iia_cell_list = []

In [7]:
def append_cell(i, cell, level, lower_bound, upper_bound, translate=True):
    if translate:
        cell.region = cell.region.translate((0.0,0.0,level), inplace=True)
    cell.region = cell.region & +lower_bound & -upper_bound
    return cell
    
for i, cells in enumerate(pin_cells):
    ia_cells, iia_cells = cells
    levels = pin_levels[i]
    lower_bounds = level_bounds[i]
    upper_bounds = level_bounds[i+1]
    for j, ia_cell in enumerate(ia_cells):
        translate = True
        if i == 3 and j == 1:
            translate = False
        ia_cell_list.append(append_cell(i, ia_cell, 
                                        levels[0], 
                                        lower_bounds[0], 
                                        upper_bounds[0],
                                        translate=translate))
    for j, iia_cell in enumerate(iia_cells):
        translate = True
        if i == 3 and j == 1:
            translate = False
        iia_cell_list.append(append_cell(i, iia_cell, 
                                         levels[1], 
                                         lower_bounds[1], 
                                         upper_bounds[1],
                                         translate=translate))
        
ia = openmc.Universe(cells=ia_cell_list)
iia = openmc.Universe(cells=iia_cell_list)

In [8]:
geo = openmc.Geometry()
geo.root_universe = ia
geo.export_to_xml()

In [9]:
geo = openmc.Geometry()
geo.root_universe = iia
geo.export_to_xml()

In [13]:
iam[1].region.get_surfaces()

OrderedDict([(18,
              Surface
              	ID             =	18
              	Name           =	
              	Type           =	z-cylinder
              	Boundary       =	transmission
              	Coefficients   
              x0              =	0.0
              y0              =	0.0
              r               =	1.71069),
             (5,
              Surface
              	ID             =	5
              	Name           =	minimum x
              	Type           =	x-plane
              	Boundary       =	transmission
              	Coefficients   
              x0              =	-4.953),
             (6,
              Surface
              	ID             =	6
              	Name           =	maximum x
              	Type           =	x-plane
              	Boundary       =	transmission
              	Coefficients   
              x0              =	4.953),
             (7,
              Surface
              	ID             =	7
              	Name           =	minimum y
 

In [ ]:
cone_i.cone

In [10]:
ia_plots = openmc.Plots()
iia_plots = openmc.Plots()

plot = openmc.Plot()
plot.origin=(0, 0, 10.0)
plot.width=(20, 20)
plot.pixels=(1000,1000)
plot.color_by='material'
plot.colors=colormap
plot.basis='xy'
ia_plots.append(plot)

plot = openmc.Plot()
plot.origin=(0, 0, 10.0)
plot.width=(20, 20)
plot.pixels=(1000,1000)
plot.color_by='material'
plot.colors=colormap
plot.basis='xy'
iia_plots.append(plot)

plot = openmc.Plot()
plot.origin=(0, 0, 23.0)
plot.width=(20, 20)
plot.pixels=(1000,1000)
plot.color_by='material'
plot.colors=colormap
plot.basis='xy'
ia_plots.append(plot)

plot = openmc.Plot()
plot.origin=(0, 0, 435)
plot.width=(20, 20)
plot.pixels=(1000,1000)
plot.color_by='material'
plot.colors=colormap
plot.basis='xy'
iia_plots.append(plot)

plot = openmc.Plot()
plot.origin=(0, 0, 420)
plot.width=(20, 20)
plot.pixels=(1000,1000)
plot.color_by='material'
plot.colors=colormap
plot.basis='xy'
ia_plots.append(plot)

plot = openmc.Plot()
plot.origin=(0, 0, 437)
plot.width=(20, 20)
plot.pixels=(1000,1000)
plot.color_by='material'
plot.colors=colormap
plot.basis='xy'
iia_plots.append(plot)

plot = openmc.Plot()
plot.origin=(0, 0, 439)
plot.width=(20, 20)
plot.pixels=(1000,1000)
plot.color_by='material'
plot.colors=colormap
plot.basis='xy'
ia_plots.append(plot)

plot = openmc.Plot()
plot.origin=(0, 0, 440)
plot.width=(20, 20)
plot.pixels=(1000,1000)
plot.color_by='material'
plot.colors=colormap
plot.basis='xy'
iia_plots.append(plot)

plot = openmc.Plot()
plot.origin=(0, 0, 448)
plot.width=(20, 20)
plot.pixels=(1000,1000)
plot.color_by='material'
plot.colors=colormap
plot.basis='xy'
ia_plots.append(plot)

plot = openmc.Plot()
plot.origin=(0, 0, 442)
plot.width=(20, 20)
plot.pixels=(1000,1000)
plot.color_by='material'
plot.colors=colormap
plot.basis='xy'
iia_plots.append(plot)

plot = openmc.Plot()
plot.origin=(0, 0, 450)
plot.width=(100, 100)
plot.pixels=(1000,1000)
plot.color_by='material'
plot.colors=colormap
plot.basis='xz'
ia_plots.append(plot)

plot = openmc.Plot()
plot.origin=(0, 0, 450)
plot.width=(100, 100)
plot.pixels=(1000,1000)
plot.color_by='material'
plot.colors=colormap
plot.basis='xz'
iia_plots.append(plot)

plot = openmc.Plot()
plot.origin=(0, 0, 50)
plot.width=(100, 100)
plot.pixels=(1000,1000)
plot.color_by='material'
plot.colors=colormap
plot.basis='xz'
ia_plots.append(plot)

plot = openmc.Plot()
plot.origin=(0, 0, 50)
plot.width=(100, 100)
plot.pixels=(1000,1000)
plot.color_by='material'
plot.colors=colormap
plot.basis='xz'
iia_plots.append(plot)

In [10]:
ia_plots.export_to_xml()

In [11]:
iia_plots.export_to_xml()

In [7]:
geo = openmc.Geometry([ia_fuel_outer_main])

In [8]:
geo.export_to_xml()

In [14]:
ia_fuel_outer_main.region.get_surfaces()

OrderedDict([(5,
              Surface
              	ID             =	5
              	Name           =	minimum x
              	Type           =	x-plane
              	Boundary       =	transmission
              	Coefficients   
              x0              =	-4.953),
             (6,
              Surface
              	ID             =	6
              	Name           =	maximum x
              	Type           =	x-plane
              	Boundary       =	transmission
              	Coefficients   
              x0              =	4.953),
             (7,
              Surface
              	ID             =	7
              	Name           =	minimum y
              	Type           =	y-plane
              	Boundary       =	transmission
              	Coefficients   
              y0              =	-4.953),
             (8,
              Surface
              	ID             =	8
              	Name           =	maximum y
              	Type           =	y-plane
              	Boundary       